In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV

In [2]:
def set_data(file):
    bikeshare_machine = pd.read_csv(file, 
                        parse_dates=['Start date', 'End date'])
    bikeshare_machine.drop('Unnamed: 0', 1, inplace=True)
    included_cols = ['start_station','end_station','Member Type','time_diff','season','mnth','holiday',
                     'weekday','workingday','weathersit','temp','hum','windspeed','miles',
                     'rush_hour','metro_dist','landmark_dist_start','landmark_dist_end']
    bikeshare_machine = bikeshare_machine[included_cols]
    bikeshare_machine['season'] = bikeshare_machine['season'].astype('category')
    bikeshare_machine['mnth'] = bikeshare_machine['mnth'].astype('category')
    bikeshare_machine['holiday'] = bikeshare_machine['holiday'].astype('category')
    bikeshare_machine['weekday'] = bikeshare_machine['weekday'].astype('category')
    bikeshare_machine['workingday'] = bikeshare_machine['workingday'].astype('category')
    bikeshare_machine['weathersit'] = bikeshare_machine['weathersit'].astype('category')
    bikeshare_machine['Member Type'] = bikeshare_machine['Member Type'].astype('category')
    bikeshare_machine['start_station'] = bikeshare_machine['start_station'].astype('category')
    bikeshare_machine['end_station'] = bikeshare_machine['end_station'].astype('category')
    bikeshare_machine['rush_hour'] = bikeshare_machine['rush_hour'].astype('category')
    col_names = ['start_station', 'end_station','member_type','time_diff','season','month','holiday',
             'weekday','work_day','weather_cat','temperature','humidity','windspeed','miles','rush_hour',
                'metro_dist','landmark_dist_start','landmark_dist_end']
    bikeshare_machine.columns = col_names
    tmin = -8
    tmax = 39
    hum_max = 100
    wind_max = 67
    bikeshare_machine['temp'] = bikeshare_machine['temperature'] * (tmax - tmin) + tmin
    bikeshare_machine['hum'] = bikeshare_machine['humidity'] * 100
    bikeshare_machine['wind'] = bikeshare_machine['windspeed'] * 67
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['rush_hour','member_type','holiday','work_day'], drop_first=True)
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['start_station','end_station','season','month','weekday','weather_cat'])
    bikeshare_machine = shuffle(bikeshare_machine)
    return bikeshare_machine

In [3]:
file = '/Users/matthewcassi/Documents/Bike-Sharing-Dataset/Bikeshare_Time_Prediction/Casual_RushMetro/landmarks.csv'
bs = set_data(file)

In [4]:
bs.columns

Index(['time_diff', 'temperature', 'humidity', 'windspeed', 'miles',
       'metro_dist', 'landmark_dist_start', 'landmark_dist_end', 'temp', 'hum',
       ...
       'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4',
       'weekday_5', 'weekday_6', 'weather_cat_1', 'weather_cat_2',
       'weather_cat_3'],
      dtype='object', length=303)

In [5]:
bs=bs.rename(columns = {'member_type_Registered':'member_type'})
bs.head()

,time_diff,temperature,humidity,windspeed,miles,metro_dist,landmark_dist_start,landmark_dist_end,temp,hum,...,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weather_cat_1,weather_cat_2,weather_cat_3
611987,8.450,0.775000,0.402917,0.183463,1.580140,0.138620,1.024951,0.595943,28.425000,40.2917,...,0,0,0,1,0,0,0,1,0,0
629131,18.033,0.768333,0.707083,0.113817,1.367495,0.351986,0.624320,0.306432,28.111651,70.7083,...,0,0,0,1,0,0,0,1,0,0
320954,23.183,0.616667,0.697083,0.342667,2.758367,0.023451,0.839915,0.485473,20.983349,69.7083,...,0,0,1,0,0,0,0,0,1,0
514571,11.517,0.684167,0.771667,0.210833,1.581563,0.148398,0.173157,0.544090,24.155849,77.1667,...,0,0,0,0,1,0,0,0,1,0
929400,7.217,0.484167,0.710000,0.205854,0.867826,0.485937,0.485473,0.346971,14.755849,71.0000,...,0,0,1,0,0,0,0,1,0,0


In [6]:
X = bs.drop('member_type', 1)
y = bs['member_type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((914885, 302), (304962, 302), (914885,), (304962,))

### GridSearch Logistic Regression

In [ ]:
c_space = np.linspace(0.01, 30, 20)
class_weight = [{0:.55, 1:.45}, {0:.6, 1:.4}, {0:.5, 1:.5}]
param_grid = {'C': c_space, 'class_weight':class_weight}

svc1 = LinearSVC(dual=False)
svc_gs1 = RandomizedSearchCV(svc1, param_grid, cv=3, n_iter=20, scoring='roc_auc')
svc_gs1.fit(X_train, y_train)

In [ ]:
pred1 = svc_gs1.predict(X_test)
score1 = svc_gs1.score(X_test, y_test)
train_score1 = svc_gs1.score(X_train, y_train)

In [ ]:
score1, train_score1

In [ ]:
confusion_matrix(y_test, pred1)

In [ ]:
print(classification_report(y_test, pred1))